In [2]:
import plotly.express as px
import pandas as pd
from datetime import date
import datetime
from dateutil import tz
import json
import os
import json



# global variables, the value is current file path
BASE_PATH=os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [3]:
remove_list = ["README.md", ".gitignore"]
path = BASE_PATH + "/../COVID-19/csse_covid_19_data/csse_covid_19_time_series"

#Read detailed US Confirmed CSV data
us_confirmed_df = pd.read_csv(path + "/time_series_covid19_confirmed_US.csv")

# Observe data
us_confirmed_df.head()


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1345,1349,1355,1371,1377,1383,1385,1398,1413,1420
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4458,4495,4513,4542,4569,4586,4609,4639,4678,4722
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,617,614,616,616,616,617,617,618,618,618
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,541,542,545,550,555,557,562,564,566,574
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,1036,1037,1049,1062,1065,1070,1071,1082,1099,1109


In [4]:
#Restructure US Confirmd using pd.melt

col_used = list(us_confirmed_df.columns)[11:] + ['Lat','Long_','Combined_Key','Province_State']
us_confirmed_gathered_df = pd.melt(us_confirmed_df[col_used],id_vars=['Lat','Long_','Combined_Key','Province_State'])
        
us_confirmed_gathered_df.head()

,Lat,Long_,Combined_Key,Province_State,variable,value
0,32.539527,-86.644082,"Autauga, Alabama, US",Alabama,1/22/20,0
1,30.727750,-87.722071,"Baldwin, Alabama, US",Alabama,1/22/20,0
2,31.868263,-85.387129,"Barbour, Alabama, US",Alabama,1/22/20,0
3,32.996421,-87.125115,"Bibb, Alabama, US",Alabama,1/22/20,0
4,33.982109,-86.567906,"Blount, Alabama, US",Alabama,1/22/20,0


In [ ]:
def save_location_json(df, file_name):
    # Loop through global confirmed df, Save Long&Lat information for each Province/State
    # Loop through US confirmed df, Save Long&Lat information for each City 
    data_dict = {}
    if 'global' in file_name:
        for _ , row in df.iterrows():   
            data_dict[row['Province/State']] = [row['Long'], row['Lat']]
    elif 'US' in file_name:
        for _ , row in df.iterrows(): 
            data_dict[row['Combined_Key']] = [row['Long_'], row['Lat']]
    
    # Save data to local as json file
    with open(file_name + ".json", "w")as f:
        f.write(json.dumps(data_dict))

#Process US add df
save_location_json(us_confirmed_gathered_df, "US_location")

In [ ]:
#Generate US Plotly Bubble Map


headerList = list(us_confirmed_gathered_df.head().columns.values)

fig = px.scatter_geo(us_confirmed_gathered_df, lat="Lat",lon="Long_",color = "Province_State",
                     hover_name="Combined_Key", animation_frame="variable", size="value", projection = "albers usa")
fig.show()

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot